__Setup IQS Connection__

In [ ]:
import iqs_jupyter
iqs = iqs_jupyter.IQSConnectorWidget(defaultValue='127.0.0.1:47700/api', auto_display=False).connect()

In [ ]:
# one can already invoke operations on the server, e.g.
iqs.mms_repository.update_mms_repository()

__Select and load MMS Commit__

In [ ]:
commit_selector = iqs.jupyter_tools.mms_commit_selector_widget(
 initial_org     = '6384a103-766c-46e0-830d-8a3b1f479479',
 initial_project = 'PROJECT-bef4f459-5d90-41fb-bc86-4f6d4ebd2dfd',
 initial_ref     = 'master',
 initial_commit  = '560d3959-3912-434a-a914-8d039d3c9a06'
)

In [ ]:
model = commit_selector.value().to_model_compartment()
model

In [ ]:
iqs.persistent_index.index_model_compartment(model)

In [ ]:
iqs.in_memory_index.load_model_compartment(model)

__Register and Execute Query__

In [ ]:
iqs.queries.unregister_all_queries()

In [ ]:
iqs.queries.register_queries_plain_text('''
pattern dependencies(source : NamedElement, target : NamedElement) {
  Dependency.supplier(dependency, target); Dependency.client(dependency, source);
}
''', query_package="iqs4twc.queries.example")

In [ ]:
qResults = iqs.query_execution.execute_query_on_model_compartment({
  "modelCompartment": model,
  "queryFQN": "iqs4twc.queries.example.dependencies"
})

__Process query results__

In [ ]:
qResults

In [ ]:
qResults.to_list_of_matches()

In [ ]:
first_result_element = qResults.to_list_of_matches()[0]['source']
first_result_element # check element hover for element descriptor details

In [ ]:
some_element = model.get_element_in_compartment_by_id("_10_0_8740266_1128086453859_950603_236") # can also look up directly
some_element

In [ ]:
from iqs_jupyter import binding
qResults_restricted = iqs.query_execution.execute_query_on_model_compartment({
  "modelCompartment": model,
  "queryFQN": "iqs4twc.queries.example.dependencies",
  "parameterBinding": binding(target=some_element)
})
qResults_restricted

__Convert query results to Python/Jupyter-friendly Pandas dataframes and process them as such__

In [ ]:
qResults_df = qResults.to_data_frame()
qResults_df

In [ ]:
import cufflinks as cf
cf.go_offline()

In [ ]:
qResults_df.applymap(lambda element: element.relative_element_id).source.value_counts().iplot(
    kind='bar', filename='cufflinks/categorical-bar-chart/frequent_sources',
    yTitle='Number of Dependencies', title='Dependencies from Source',
)

__Sandbox__

In [ ]:
commit_selector.value()